**[Question 4]  Leveraging a Pretrained Model for CIFAR-10 (10 marks)**

** optimizer2=tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9) undecided how to use this or insert where **

> Objective: Use a pretrained model from TensorFlow's Keras applications  **as a feature extractor or for fine-tuning on the CIFAR-10 dataset**, and a**chieve >90% accuracy on the test dataset** of CIFAR-10. 

In [3]:
#import the libraries and packages needed
import tensorflow as tf
from tensorflow.keras import models, layers, optimizers

ModuleNotFoundError: No module named 'tensorflow'

> **Requirement**: Dataset Preparation: Load the CIFAR-10 dataset, applying normalization and any required preprocessing to match the input format of the pretrained model.

In [4]:
def preprocess_data(X, Y, num_class=10):
    """
    returns preprocessed data
    by using the resnet50 preprocess_input function which normalizes the input data and
    converts the labels to one-hot encoding using the to_categorical function with 10 classes
    """
    #X_resized = tf.image.resize(X, (224, 224))
    X_p = tf.keras.applications.resnet50.preprocess_input(X)
    Y_p = tf.keras.utils.to_categorical(Y, num_class)
    return X_p, Y_p

In [5]:
# Load the CIFAR-10 dataset
(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.cifar10.load_data()

# preprocess data using the preprocess_data function
#''''
X_train_p, Y_train_p = preprocess_data(X_train, Y_train)
X_test_p, Y_test_p = preprocess_data(X_test, Y_test)
#'''

170498071/170498071 [==============================] - 13s 0us/step


In [6]:
# sanity check of the shape of the preprocessed data, datapoints statistics,
# and labels if its turned into one-hot encoding
print(X_train_p.shape, Y_train_p.shape)
print(X_test_p.shape, Y_test_p.shape)
print('number of classes:',num_class:=Y_train_p.shape[1])
print('first 5 labels:\n', Y_train_p[:5],'\n')
print('max, and min of the datapoints:', X_train_p.max(), X_train_p.min())

(50000, 32, 32, 3) (50000, 10)
(10000, 32, 32, 3) (10000, 10)
number of classes: 10
first 5 labels:
 [[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]] 

max, and min of the datapoints: 151.061 -123.68


> **Requirement**: Model Selection and Modification: Choose a pretrained model (e.g., ResNet, VGG16) and modify it for CIFAR-10 classification. Describe how you adapt the model for the 10 classes of CIFAR-10 (e.g., modifying the top layer, adjusting input size).

> To use ResNet50 pretrained on ImageNet dataset as a feature extractor
- To **remove** the final layer of ResNet50 that is used to classify for the ImageNet data
- To add a dense layer with 10 classes with softmax activations because the original classifier has 1000 classes (from ImageNet)
- To freeze all the prior layers of ResNet50, and perform transfer learning on the layer(s) added to adapt to learning how to classify the CIFAR10 images 
- Stacking: Add upscaling of CIFAR10 images (32x32x3) to match the spatial dimension of ResNet50 (224x224x3) as it was trained on ImageNet 224x224x3 images -> ResNet50 as base model -> added layer(s)

In [7]:
# Load ResNet50 model without the top layer
base_model = tf.keras.applications.ResNet50(include_top=False, 
                                            weights='imagenet', 
                                            input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze ResNet50 layers

94765736/94765736 [==============================] - 5s 0us/step


> **Model 1 architecture**: Upscaling -> Base Model (ResNet50) -> Flatten -> Dense layer with Softmax Activation (Classifier)

In [ ]:
# Build the model1
model1 = models.Sequential([
    # Upscale images
    layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear"),
    # Add the pre-trained ResNet50 base model
    base_model,
    # Add custom layers on top
    layers.Flatten(),
    layers.Dense(10, activation='softmax')  # For CIFAR-10 classes
])

In [ ]:
# Compile the model
model1.compile(optimizer=optimizers.SGD(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Describe the model architecture and params for sanity check
model1.summary()

In [ ]:
# Train the model
model.fit(X_train, Y_train_p, batch_size=128, epochs=5,
          validation_data=(X_test, Y_test_p))

> **Model1 Evaluation**: Observed that the model is overfitting quite badly from epoch 3 onwards as the gap between train acc and test acc is widening. Upon research, decided to use Global Average Pooling (GAP)to find the precense of a global feature across all 3 channels, instead of purely just flattening before feeding it into the classifier (softmax). As a result of reducing the number of parameters, this can help prevent overfitting

> Reference: https://stackabuse.com/dont-use-flatten-global-pooling-for-cnns-with-tensorflow-and-keras/ , https://paperswithcode.com/method/global-average-pooling#:~:text=Global%20Average%20Pooling%20is%20a%20pooling%20operation%20designed,the%20classification%20task%20in%20the%20last%20mlpconv%20layer.

> **Model2 Architecture**: Upscaling -> Base Model (ResNet50) -> Global Average Pooling -> Dense layer with Softmax Activation (Classifier)

In [ ]:
# Build the model
model2 = models.Sequential([
    # Upscale images
    layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear"),
    # Add the pre-trained ResNet50 base model
    base_model,
    # Add custom layers on top
    layers.Global Average Pooling,
    layers.Dense(10, activation='softmax')  # For CIFAR-10 classes
])

In [ ]:
# Compile the model
model2.compile(optimizer=optimizers.SGD(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model2.summary()

In [ ]:
# Train the model
# will just start small with 5 epochs
model2.fit(X_train, Y_train_p, batch_size=128, epochs=5,
          validation_data=(X_test, Y_test_p))

> **Model 2 evauation** : Improved performance with using GAP but unable to hit 90%. Decided to increase a few more hidden layers with batch norm and softmax activation like what we did on CNN. Batch norm is added and applied immediately after the dense layer to increases training stability and improves convergence.

> **Model 3 architecture**: Upscaling > Base Model (Resnet50) > Dense layer of 512 neurons -> Batch normalisation -> Relu activation -> Dense layer of 128 neurons -> Batch normalisation -> Relu activation -> Dense layer with Softmax Activation (Classifier)

In [8]:
# Build the model
model = models.Sequential([
    # Upscale images
    layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear"),
    # Add the pre-trained ResNet50 base model
    base_model,
    # Add custom layers on top
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')  # For CIFAR-10 classes
])

In [9]:
# Compile the model
model.compile(optimizer=optimizers.SGD(lr=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# Train the model
# will just start small with 5 epochs
model.fit(X_train, Y_train_p, batch_size=128, epochs=5,
          validation_data=(X_test, Y_test_p))

Epoch 1/5
391/391 [==============================] - 194s 464ms/step - loss: 0.9557 - accuracy: 0.7312 - val_loss: 0.4907 - val_accuracy: 0.8352
Epoch 2/5
391/391 [==============================] - 184s 471ms/step - loss: 0.4359 - accuracy: 0.8529 - val_loss: 0.4059 - val_accuracy: 0.8607
Epoch 3/5
391/391 [==============================] - 187s 480ms/step - loss: 0.3198 - accuracy: 0.8914 - val_loss: 0.3880 - val_accuracy: 0.8713
Epoch 4/5
391/391 [==============================] - 188s 482ms/step - loss: 0.2382 - accuracy: 0.9182 - val_loss: 0.3985 - val_accuracy: 0.8717
Epoch 5/5
391/391 [==============================] - 192s 492ms/step - loss: 0.1844 - accuracy: 0.9374 - val_loss: 0.3987 - val_accuracy: 0.8782


*stop* here ------

In [19]:
# Build the model2; improved version by adding batch normalization to adjust the activations with a mean of 0 and std dev of 1
# batch norm added AFTER activations. source:https://www.deeplearningbook.org/contents/optimization.html
model2 = models.Sequential([
    # Upscale images
    layers.experimental.preprocessing.Resizing(224, 224, interpolation="bilinear"),
    base_model,
    layers.BatchNormalization(),  # Add Batch Normalization after ResNet50
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),  # Add Batch Normalization after the first Dense layer
    layers.Dropout(0.2),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),  # Add Batch Normalization after the second Dense layer
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')  # Output layer for CIFAR-10 classes
])

In [20]:
# Compile the model
# for SGD, learning rate decay is used to gradually reduces the learning rate during training,
# allowing the model to fine-tune and adjust weights with greater precision as it approaches convergence
optimizer2=tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9)
model2.compile(optimizer=optimizer2,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model2.fit(X_train_p, Y_train_p, batch_size=128, epochs=5,
          validation_data=(X_test_p, Y_test_p))

Epoch 1/5
391/391 [==============================] - 204s 506ms/step - loss: 0.7420 - accuracy: 0.7579 - val_loss: 0.3960 - val_accuracy: 0.8698
Epoch 2/5
391/391 [==============================] - 182s 466ms/step - loss: 0.3616 - accuracy: 0.8826 - val_loss: 0.3480 - val_accuracy: 0.8865
Epoch 3/5
391/391 [==============================] - 195s 499ms/step - loss: 0.2818 - accuracy: 0.9105 - val_loss: 0.3231 - val_accuracy: 0.8954
Epoch 4/5
391/391 [==============================] - 184s 471ms/step - loss: 0.2285 - accuracy: 0.9307 - val_loss: 0.3087 - val_accuracy: 0.8991
Epoch 5/5
391/391 [==============================] - 195s 499ms/step - loss: 0.1915 - accuracy: 0.9433 - val_loss: 0.2990 - val_accuracy: 0.9015
